<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Losses

Goal : have a table that compares the different losses on MNIST.

Data : MNIST

Models : AttnNPFXL and ConvNPFXL for CNPFLoss, NLLLloss, ISLoss, ElboLoss

Runs : 5

In [1]:
import os

os.chdir("..")

import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [2]:
def load_reeval(name, n_test, bs=16, is_iw=False):
    name = f"{name}_{str(n_test)}"
    if is_iw:
        name += "_iw"
        
    args = get_exp_args(name, is_load=False, is_reeval=True)
    args = [arg for arg in args if (arg.dataset == "mnist" and arg.runs <= 3)]
    for arg in args:
        arg.n_z_samples_test = n_test
        arg.batch_size=bs
        if is_iw:
            arg.is_iw_eval=True
    return args

In [3]:
args=load_reeval("exp_losses_isglobal", 128, bs=8)

In [5]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24,  
                           cpus_per_task=10, 
                           mem='32GB',
                           partition="dev",
                           comment="neurips",
                          )

In [6]:
jobs = executor.map_array(Run(), args)

In [7]:
############################################################

In [2]:
jobs

NameError: name 'jobs' is not defined

In [15]:
for j in jobs:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2020-06-04 03:15:15,739) - Starting with JobEnvironment(job_id=27221377_0, hostname=learnfair0837, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-06-04 03:15:15,739) - Loading pickle: /private/home/yannd/projects/NPF/logs/27221377_0/27221377_0_submitted.pkl

--- Loading mnist/ConvNPFXL_NllLNPF/run_0 ---

submitit ERROR (2020-06-04 03:15:27,225) - Submitted job triggered an exception

--------------------------------
submitit INFO (2020-06-04 03:15:15,739) - Starting with JobEnvironment(job_id=27221377_1, hostname=learnfair0837, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-06-04 03:15:15,739) - Loading pickle: /private/home/yannd/projects/NPF/logs/27221377_1/27221377_1_submitted.pkl

--- Loading mnist/ConvNPFXL_NllLNPF/run_1 ---

submitit ERROR (2020-06-04 03:15:26,965) - Submitted job triggered an exception

--------------------------------
submitit INFO (2020-06-04 03:15:15,918) - Starting with Job

In [16]:
for j in jobs:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit ERROR (2020-06-04 03:15:27,225) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 186, in submitit_main
    process_job(args.folder)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 179, in process_job
    raise error
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submi

In [17]:
for job in jobs:
    job.cancel()